In [1]:
%matplotlib inline
import os
from subprocess import call

import xarray as xr
import numpy as np
import cftime
import dask

from scipy import signal

from itertools import product

import cartopy
import cartopy.crs as ccrs
import  matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

import geotools as gt
import plottools as pt
import project as P

USER = os.environ['USER']

In [2]:
xr_open_ds = {'chunks' : {'time':1},
              'decode_coords' : False,
              'decode_times' : False}
xr.set_options(enable_cftimeindex=True)

year_range = np.array((249,316))+1699
time = [cftime.DatetimeNoLeap(year, month, 1) for year, month in 
                  product(range(year_range[0], year_range[1]+1), range(1, 13))]
ypm = np.array([31,28,31,30,31,30,31,31,30,31,30,31])/365
wgt = xr.DataArray(np.tile(ypm,int(np.diff(year_range)+1)),dims=('time'),coords={'time':time})

np.testing.assert_allclose(wgt.groupby('time.year').sum(),1.)

In [3]:
from dask.distributed import Client
from dask_jobqueue import PBSCluster

dask.config.set({'distributed.dashboard.link':'http://localhost:{port}/status'})

# Lots of arguments to this command are set in ~/.config/dask/jobqueue.yaml
cluster = PBSCluster(queue='regular',
                     cores = 36,
                     processes = 9,
                     memory = '100GB',                     
                     project = 'NCGD0033',
                     walltime = '04:00:00',
                     local_directory=f'/glade/scratch/{USER}/dask-tmp')
client = Client(cluster)

In [4]:
Nnodes = 4
cluster.scale(9*Nnodes)

Read data and compute vertical means within thermocline depth range

In [5]:
clobber = False
file_out = f'{P.dirt}/work.fosi_on_sigma.zarr'

variables = ['O2','PD','UVEL','VVEL','TEMP','SALT']

if clobber:
    call(['rm','-fr',file_out])

if os.path.exists(file_out):
    ds = xr.open_zarr(file_out,decode_times=False,decode_coords=False)

else:
    droot = '/glade/p_old/decpred/CESM-DPLE_POPCICEhindcast'

    ds = xr.Dataset()
    for v in variables:
        ds = xr.merge((ds,xr.open_dataset(f'{droot}/g.e11_LENS.GECOIAF.T62_g16.009.pop.h.{v}.024901-031612.nc',**xr_open_ds)))

    ds = ds.isel(nlat=slice(187,331),nlon=slice(137,276))
    
    ds = P.interp_to_pd(ds)        
    print(f'writing {file_out}')
    ds.to_zarr(file_out)

ds.time.values = time   
year = np.array([d.year for d in ds.time.values])
yearfrac = year + np.tile(np.cumsum(ypm),np.diff(year_range)+1)

grid_vars = [v for v in ds.variables if 'time' not in ds[v].dims]+['time_bound']

Compute monthly anomalies

In [6]:
nx = np.where((P.year_range_clim.start <= year) & (year <= P.year_range_clim.stop))[0]
dsm = ds.drop(grid_vars).isel(time=nx).groupby('time.month').mean('time')
dsyc = ds.groupby('time.month') - dsm
dsyc = xr.merge((dsyc,ds.drop(variables))) # put the grid_vars back

Compute annual means

In [7]:
#dsy = (ds.drop(grid_vars) * wgt).groupby('time.year').sum('time')
#dsy = xr.merge((dsy,ds.drop([v for v in ds.variables if v not in grid_vars])))

Compute anomaly field

In [8]:
dsyc = dsyc.isel(sigma=0)

Compute regional mean

In [9]:
dsyr = P.regional_mean(dsyc)
dsyr = dsyr.compute()

Make a plot

In [10]:
ds['UVEL'] = ds.UVEL.where((ds.UVEL!=0)&(ds.VVEL!=0))
ds['VVEL'] = ds.VVEL.where((ds.UVEL!=0)&(ds.VVEL!=0))

In [11]:
old_err_settings = np.seterr(invalid='ignore')

extent = [120,260,10,65]
prj = ccrs.Mercator(central_longitude=np.mean(extent[0:2]),
                    min_latitude=extent[2],
                    max_latitude=extent[3])
cf_specsa = {'levels':np.arange(-20,25,5),
            'cmap':'PRGn',
            #'norm':norm_o2,
            'extend':'both'} 

rx,ry = P.region_box()

norm_o2 = pt.MidPointNorm(midpoint=50.)
cf_specs = {'levels':[0,10,20,30,40,50,60,80,100,125,150,175,200,225,250,275,300],
            'cmap':'PRGn',
            'norm':norm_o2,
            'extend':'both'}
dirf = 'fig/n-pac-movie-26_5'
if not os.path.exists(dirf):
    call(['mkdir','-p',dirf])

#-- loop over time
for i,d in enumerate(ds.time):
    print(f'{i+1}/{len(ds.time)} ',end='')
    if i < 125: continue
        
    #-- canvas
    fig = plt.figure(figsize=(12,6))
    gs = gridspec.GridSpec(6, 8)

    #-------------------------------------------------
    #-- Axes 1: full field
    ax = fig.add_subplot(gs[0:4,0:4],projection=prj)    
    ax.set_extent(extent)    
    lon,lat,field = pt.adjust_pop_grid(ds.TLONG.values,ds.TLAT.values,ds['O2'].isel(sigma=0,time=i).values)
    lon,lat,uvel = pt.adjust_pop_grid(ds.TLONG.values,ds.TLAT.values,ds['UVEL'].isel(sigma=0,time=i).values)
    lon,lat,vvel = pt.adjust_pop_grid(ds.TLONG.values,ds.TLAT.values,ds['VVEL'].isel(sigma=0,time=i).values)
     
    
    cf = ax.contourf(lon,lat,field,**cf_specs,transform=ccrs.PlateCarree())    
    sp = ax.streamplot(lon,lat,uvel,vvel,
                   linewidth=0.2,
                   arrowsize = 0.25,
                   density=5,
                   color='k',
                   transform=ccrs.PlateCarree())
    ax.plot(rx,ry,marker=None,linewidth=1,color='#d95f02',transform=ccrs.PlateCarree())
    land = ax.add_feature(cartopy.feature.NaturalEarthFeature('physical','land','110m',
                  edgecolor='face',
                  facecolor='gray'),zorder=100)

    cb = plt.colorbar(cf,shrink=0.65)   
    ax.set_title('O$_2$ concentration',loc='center')
    cb.ax.set_title('mmol m$^{-3}$',loc='left')

    #-------------------------------------------------    
    #-- Axes 2: anomaly field
    ax = fig.add_subplot(gs[0:4,4:8],projection=prj)    
    ax.set_extent(extent)
    
    lon,lat,field = pt.adjust_pop_grid(dsyc.TLONG.values,dsyc.TLAT.values,dsyc['O2'].isel(time=i).values)
    cf = ax.contourf(lon,lat,field,**cf_specsa,transform=ccrs.PlateCarree())
    
    #lon,lat,uvel = pt.adjust_pop_grid(ds.TLONG.values,ds.TLAT.values,ds['UVEL'].isel(time=i).values)
    #lon,lat,vvel = pt.adjust_pop_grid(ds.TLONG.values,ds.TLAT.values,ds['VVEL'].isel(time=i).values)
    sp = ax.streamplot(lon,lat,uvel,vvel,
                   linewidth=0.2,
                   arrowsize = 0.25,
                   density=5,
                   color='k',
                   transform=ccrs.PlateCarree())
    
    ax.plot(rx,ry,marker=None,linewidth=1,color='#d95f02',transform=ccrs.PlateCarree())
    land = ax.add_feature(cartopy.feature.NaturalEarthFeature('physical','land','110m',
                  edgecolor='face',
                  facecolor='gray'),zorder=100)
    
    cb = plt.colorbar(cf,shrink=0.65)
    ax.set_title('O$_2$ anomaly',loc='center')
    cb.ax.set_title('mmol m$^{-3}$',loc='left')

    #-------------------------------------------------    
    #-- Axes 3: time locator
    ax = fig.add_subplot(gs[4:,1:7])
    ax.plot(yearfrac,dsyr.O2,color='#377eb8')
    ax.axhline(0,color='k',linewidth=0.5,zorder=-10)
    ax.axvline(yearfrac[i],color='#d95f02')
    ax.set_ylabel('O$_2$ anomaly [mmol m$^{-3}$]')

    plt.savefig(f'{dirf}/{i:03d}.png',dpi=300,bbox_inches='tight')
    plt.close()
    
np.seterr(**old_err_settings)

1/816 2/816 3/816 4/816 5/816 6/816 7/816 8/816 9/816 10/816 11/816 12/816 13/816 14/816 15/816 16/816 17/816 18/816 19/816 20/816 21/816 22/816 23/816 24/816 25/816 26/816 27/816 28/816 29/816 30/816 31/816 32/816 33/816 34/816 35/816 36/816 37/816 38/816 39/816 40/816 41/816 42/816 43/816 44/816 45/816 46/816 47/816 48/816 49/816 50/816 51/816 52/816 53/816 54/816 55/816 56/816 57/816 58/816 59/816 60/816 61/816 62/816 63/816 64/816 65/816 66/816 67/816 68/816 69/816 70/816 71/816 72/816 73/816 74/816 75/816 76/816 77/816 78/816 79/816 80/816 81/816 82/816 83/816 84/816 85/816 86/816 87/816 88/816 89/816 90/816 91/816 92/816 93/816 94/816 95/816 96/816 97/816 98/816 99/816 100/816 101/816 102/816 103/816 104/816 105/816 106/816 107/816 108/816 109/816 110/816 111/816 112/816 113/816 114/816 115/816 116/816 117/816 118/816 119/816 120/816 121/816 122/816 123/816 124/816 125/816 126/816 127/816 128/816 129/816 130/816 131/816 132/816 133/816 134/816 135/816 136/816 137/816 138/816 139/

TopologyException: side location conflict at -186.29415103694413 39.528982139802459


TopologicalError: The operation 'GEOSContains_r' could not be performed. Likely cause is invalidity of the geometry <shapely.geometry.polygon.Polygon object at 0x2aacf701def0>

TopologyException: side location conflict at -186.29415103694413 39.528982139802459


Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x2aab6beef9d8> (for post_execute):


TopologicalError: The operation 'GEOSContains_r' could not be performed. Likely cause is invalidity of the geometry <shapely.geometry.polygon.Polygon object at 0x2ab003e20f98>

TopologyException: side location conflict at -186.29415103694413 39.528982139802459


TopologicalError: The operation 'GEOSContains_r' could not be performed. Likely cause is invalidity of the geometry <shapely.geometry.polygon.Polygon object at 0x2aacf525ae10>

<Figure size 864x432 with 3 Axes>